In [198]:
import pandas as pd
import csv

In [199]:
tabla_series = pd.read_csv("./components/info_gustos_series.csv", encoding="utf-8")
tabla_movies = pd.read_csv("./components/info_gustos_peliculas.csv", encoding="utf-8")
tabla_musica = pd.read_csv("./components/info_gustos_canciones.csv", encoding="utf-8")
tabla_libros = pd.read_csv("./components/info_gustos_libros.csv", encoding="utf-8")
tabla_juegos = pd.read_csv("./components/info_gustos_videojuegos.csv", encoding="utf-8")
tabla_series.columns = ["Nombre", "Carrera", "Edad", "Series", "Genre"]
tabla_movies.columns = ["Nombre", "Carrera", "Edad", "Pelis", "Genre"]
tabla_musica.columns = ["Nombre", "Carrera", "Edad", "Musica", "Artista", "Genre"]
tabla_libros.columns = ["Nombre", "Carrera", "Edad", "Libro", "Genre"]
tabla_juegos.columns = ["Nombre", "Carrera", "Edad", "Juego", "Genre"]

peliculas = tabla_movies["Pelis"]
series = tabla_series["Series"]
musica = tabla_musica[["Musica", "Artista"]]
libros = tabla_libros["Libro"]
videojuegos = tabla_juegos["Juego"]
for i, k in musica.iterrows():
  print(k.values[0], k.values[1])
  
# videojuegos.pop(0)
# print(videojuegos)
# print(libros)
# print(musica)

Amapolas  Natanael-Cano
Outside Calvin-Harris 
Presidente  GB-Luis R-natannael-cano 
Cabaret Cabaret-ensembly
Chant-Repriesed Hadestown-Ensembly
Too-Sweet Hozier
Vamp! Some-Like-it-Hot-Cast
Car-Radio Twenty-One-Pilots
Cinderella Mac-Miller
Narcisita  The-Warning
Shes-my-collar Gorillaz
Razzmatazz IDKHBTFM
La-Barca Luis-Miguel
If-it-aint-me Dua-Lipa
Nocturne-Op.-9-No.-2 Federic-Chopin
Somebody-to-Love Justin-Bieber
El reloj Luis-Miguel
And-I-love-her The-Beatles
The-Long-And-Winding-Road The-Beatles
Yesterday The-Beatles
Cien-Años Pedro-Infante
Blackbird The-Beatles
Si-Nos-Dejan Luis-Miguel
One-and-only nan
Sweetest-Devotion nan
All-I-Ask nan
I'll-be-waiting nan
Take-it-all nan
Pas-de-deux P.I.Tchaïkovsky
Torn Natalie-Imbruglia
En-la-ciudad-de-la-furia Soda-Estéreo
L'Amour-Toujours Gigi-DÁngostino
Good-luck-babe Chapelle-Roan
Je-te-lasserai-des-mots Patrick-Watson
Holding-on-to-you Twenty-one-pilots
Iris Goo-goo-dolls
Hard-feelings Lorde
Gotta-go-home Boney-M
Tu con el Frankie-Ruiz
Sabo

In [200]:
# Displaying final output format
from IPython.display import display, Markdown, Latex
# LangChain Dependencies
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
# from langchain_community.chat_models import ChatOllama
from langchain_ollama.llms import OllamaLLM
from langgraph.graph import START, END, StateGraph
# For State Graph 
from typing_extensions import TypedDict
import json
import os

In [201]:
# Defining LLM
local_llm = 'llama3'
llama3 = OllamaLLM(model=local_llm, temperature=0)
llama3_json = OllamaLLM(model=local_llm, format='json', temperature=0)

In [202]:
# Generation Prompt

media_analizer = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert in all tv shows and movies, you will be given an array of may tv shows, and your job is to return
    a your response with the format JSON: {{"NameOfTheShow": "genre"}} with only one genre for each show.
    Do not give any further explanation on your response.
    Try using general purpose genres, for example, sitcom and comedy are the same.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Shows: {shows} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["shows"],
)

series_chain = media_analizer | llama3_json | JsonOutputParser()


In [203]:
videogame_analizer = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert in all videogames, you will a videogame, and your job is to return
    a your response with the format JSON: {{"NameOfTheGame": "genre"}} with only one genre for each game.
    Do not give any further explanation on your response.
    Try using general purpose genres, for example, sitcom and comedy are the same.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Shows: {videogame} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["videogame"],
)

series_chain = videogame_analizer | llama3_json | JsonOutputParser()

In [204]:
# Generation Prompt

music_analizer = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert in all kind of music, you will be given a song, and your job is to return
    a your response with the format JSON: {{"NameOfTheSong": "genre"}} with only one genre for each song.
    Do not give any further explanation on your response.
    Try using general purpose genres, for example, sitcom and comedy are the same.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Shows: {song} 
    Artist: {artist}
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["song", "artist"],
)

music_chain = music_analizer | llama3_json | JsonOutputParser()

In [205]:
# Generation Prompt

book_analizer = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert in all kind of books, you will be given a book, and your job is to return
    a your response with the format JSON: {{"NameOfTheBook": "genre"}} with only one genre for each book.
    Do not give any further explanation on your response.
    Try using general purpose genres, for example, sitcom and comedy are the same.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Book: {book} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["book"],
)

book_chain = book_analizer | llama3_json | JsonOutputParser()

In [206]:
# Node - Generate

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    print("Step: Generating Final Response")
    shows = state["shows"]
    generation = series_chain.invoke({"shows": shows})

    # Answer Generation

    return {"generation": generation}

In [207]:
# Node - Generate

def generate_games(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    # print("Step: Generating Videogames")
    games = state["videogame"]
    generation = series_chain.invoke({"videogame": games})

    # Answer Generation

    return {"generation": generation}

In [208]:
# Node - Generate

def generate_music(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    # print("Step: Generating Final Response")
    song = state["song"]
    artist = state["artist"]
    # print(song)

    # Answer Generation
    generation = music_chain.invoke({"song": song, "artist": artist})

    return {"generation": generation}

In [209]:
def generate_books(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    # print("Step: Generating Final Response")
    book = state["book"]
    # print(book)

    # Answer Generation
    generation = book_chain.invoke({"book": book})

    return {"generation": generation}

In [210]:
# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        search_query: revised question for web search
        context: web_search result
    """
    shows: str
    song: str
    artist: str
    book: str
    media: str
    videogame: str
    generation : str

In [211]:
def router(state):
  # print(state["media"])
  return state["media"]

In [212]:
# Build the nodes
workflow = StateGraph(GraphState)
workflow.add_node("generate_shows", generate)
workflow.add_node("generate_music", generate_music)
workflow.add_node("generate_books", generate_books)
workflow.add_node("generate_videogames", generate_games)

workflow.set_conditional_entry_point(
  router,
  {
    "shows": "generate_shows",
    "music": "generate_music",
    "book": "generate_books",
    "videogames": "generate_videogames"
  }
)

workflow.add_edge("generate_shows", END)
workflow.add_edge("generate_music", END)
workflow.add_edge("generate_books", END)
workflow.add_edge("generate_videogames", END)

# Compile the workflow
local_agent = workflow.compile()


In [213]:
def run_agent_series():
    shows = []
    for serie in series:
        output = local_agent.invoke({"media": "shows", "media": "shows", "shows": serie})
        print(output["generation"])
        shows.append(output["generation"])
    return shows

def run_agent_movies():
    movies = []
    for movie in peliculas:
        output = local_agent.invoke({"media": "shows", "shows": movie})
        print(output["generation"])
        movies.append(output["generation"])

    return movies    

def run_agent_music():
    music = []
    # for song, artist in musica:
    for i, k in musica.iterrows():
        # print(k.values)
        output = local_agent.invoke({"media": "music", "song": k.values[0], "artist": k.values[1]})
        print(output["generation"])
        music.append(output["generation"])
    
    return music

def run_agent_book():
    books = []
    for book in libros:
        output = local_agent.invoke({"media": "book", "book": book})
        print(output["generation"])
        books.append(output["generation"])
    
    return books

def run_agent_videogames():
    games = []
    for game in videojuegos:
        output = local_agent.invoke({"media": "videogames", "videogame": game})
        print(output["generation"])
        games.append(output["generation"])
    return games

In [214]:
# all_shows = run_agent_series()

In [215]:
# all_movies = run_agent_movies()

In [216]:
all_music = run_agent_music()
df = pd.DataFrame([{"Song": k, "Genre": v} for s in all_music for k, v in s.items()])
print(df)
df.to_csv("dict_music.csv", index=False)

{'Amapolas': 'Latin'}
{'Outside': 'Electronic'}
{'Presidente': 'Reggaeton'}
{'Cabaret': 'Musical'}
{'Chant-Repriesed': 'Folk'}
{'Too-Sweet': 'Blues'}
{'Vamp': 'Rock'}
{'Car-Radio': 'Pop'}
{'Cinderella': 'Hip-Hop'}
{'Narcisita': 'Electronic'}
{"She's My Collar": 'Hip-Hop'}
{'Razzmatazz': 'Electronic'}
{'La Barca': 'Latin'}
{"If It Ain't Me": 'Pop'}
{'Nocturne-Op.-9-No.-2': 'Classical'}
{'Somebody to Love': 'Pop'}
{'El Reloj': 'Latin'}
{'And I Love Her': 'Pop'}
{'The Long And Winding Road': 'Rock'}
{'Yesterday': 'Rock'}
{'Cien-Años': 'Ranchera'}
{'Blackbird': 'Folk'}
{'Shows': 'Latin'}
{'One-and-only': 'Instrumental'}
{'Sweetest Devotion': 'Pop'}
{'All I Ask': 'Pop'}
{"I'll Be Waiting": 'R&B'}
{'Take-it-all': 'Electronic'}
{'Pas-de-deux': 'Classical'}
{'Torn': 'Pop'}
{'En-la-ciudad-de-la-furia': 'Rock'}
{"L'Amour-Toujours": 'Trance'}
{'Good-Luck-Babe': 'Hip-Hop'}
{'Je-te-lasserai-des-mots': 'French Pop'}
{'Holding On To You': 'Pop'}
{'Iris': 'Rock'}
{'Ella Marija Lani Owen (Lorde)': 'Ind

In [217]:
# all_books = run_agent_book()

In [218]:
# all_videogames = run_agent_videogames()
# for game in videojuegos:
#   print(game)
#   print(local_agent.invoke({"media": "videogames", "videogame": game})["generation"])

In [219]:
# df = pd.DataFrame([{"Book": k, "Genre": v} for s in all_videogames for k, v in s.items()])
# print(df)
# df.to_csv("dict_games.csv", index=False)


In [220]:
# df = pd.DataFrame([{"Show": k, "Genre": v} for s in all_shows for k, v in s.items()])
# print(df)
# df.to_csv("dict_series.csv", index=False)
# df = pd.DataFrame([{"Movie": k, "Genre": v} for s in all_movies for k, v in s.items()])
# print(df)
# df.to_csv("dict_movies.csv", index=False)

In [221]:
#References:
# Pandas: https://pandas.pydata.org/docs/getting_started/index.html
# Langchain: https://langchain-ai.github.io/langgraph/tutorials/workflows/#prompt-chaining 